In [21]:
import os
import json
import tweepy
import pandas as pd

In [2]:
# Leitura das chaves
with open("keys.txt","r") as text:
    text = text.read()
    keys = json.loads(text)

print(keys.keys())

dict_keys(['API_KEY', 'API_KEY_SECRET', 'ACCESS_TOKEN', 'ACCESS_TOKEN_SECRET', 'BEARER_TOKEN'])


In [3]:
def conect_api():
    auth = tweepy.OAuth1UserHandler(
      keys['API_KEY'], 
      keys['API_KEY_SECRET'], 
      keys['ACCESS_TOKEN'], 
      keys['ACCESS_TOKEN_SECRET'])
    
    return tweepy.API(auth)

In [4]:
api = conect_api()

## capturando tweets com o uso de cursores

#### LISTAGEM DE RESUATRANTES

In [8]:
restaurantes = {"mcdonalds","méqui","burger king", "girafas", "subway","madero",
                "starbucks", "pizza hut", "dominos","habibs","outback","KFC","Taco Bell"}

### Função para minerar os dados

In [36]:
def data_mining(sentimento):
    
    sentimento = sentimento.lower();
    publicacoes = []
    
    if sentimento == "positivo":
        sentimentos = {"😍", "🥰", "😋","🤩","😀","😇"}
    
    elif sentimento == "negativo":
        sentimentos = {"🤮","🤢","🤬","😡","😠","💩","😣"}
    
    else:
        raise ValueError("Parametro incorreto!")
    
    
    for restaurante in restaurantes:
        for emoji in sentimentos:
            query = f"{restaurante} AND {emoji}"
            
            try:
                for status in tweepy.Cursor(api.search_tweets, q = query, lang="pt",
                                            count=100).items(10000):
                    publicacoes.append(status)
            except:
                print("limite de requisicao estourado!")
    
    return publicacoes

### Mineração

In [14]:
publicacoes_positivas = data_mining("positivo");

In [37]:
publicacoes_negativas = data_mining("negativo");

In [55]:
# for pub in publicacoes_positivas:
#     print(pub.text)
#     print(pub.created_at)
#     print("-------")

In [54]:
# for pub in publicacoes_negativas:
#     print(pub.text)
#     print(pub.created_at)
#     print("-------")

In [39]:
len(publicacoes_positivas), len(publicacoes_negativas)

(729, 111)

### Função para criar os DataFrames

In [22]:
def cria_data_frame(dados: list) -> pd.DataFrame:
    
    id_autor, texto, data_publicacao = [], [], []

    for pub in dados:
        id_autor.append(pub.id_str)
        texto.append(pub.text)
        data_publicacao.append(pub.created_at)
    
    main_data = pd.DataFrame({"id_autor": id_autor, "texto": texto ,
                           "data_publicacao": data_publicacao })
    
    return main_data

### transformando em DataFrame

In [40]:
df_positivo = cria_data_frame(publicacoes_positivas)
df_negativo = cria_data_frame(publicacoes_negativas)

In [51]:
df_positivo.head()

,id_autor,texto,data_publicacao
0,1534956288387473410,"nunca gostei do habibs, e na única vez que ten...",2022-06-09 17:51:11+00:00
1,1533459101765799942,O auge pra mim foi ter habibs no Castelão onte...,2022-06-05 14:41:54+00:00
2,1532768987687309320,Adoraria experimentar pena que eu não tenho um...,2022-06-03 16:59:38+00:00
3,1532565023289131008,@_thecria O habibs é na esquina 😍,2022-06-03 03:29:09+00:00
4,1532541435660091400,"Ganhei esfiha do habibs,chocolate amo y amo😍😋",2022-06-03 01:55:26+00:00


In [50]:
df_negativo.head()

,id_autor,texto,data_publicacao
0,1532385165275701248,@Oliver4Arthur Eu to te esperando faz 2h aqui ...,2022-06-02 15:34:28+00:00
1,1532155314887327744,Porra deixaram nós sem habibs msm 😡 https://t....,2022-06-02 00:21:07+00:00
2,1532131862323265541,@DerthWalter @trans_gabsdias É um absurdo habi...,2022-06-01 22:47:56+00:00
3,1532050520516579328,@Coltro777 tiraram o nosso habibs da gnt😡,2022-06-01 17:24:42+00:00
4,1533172860201619457,“Hambúrguer de esfiha do Habibs”!?\n🤢🤮🤮🤮🤮,2022-06-04 19:44:29+00:00


### salvando os dados

In [49]:
df_negativo["data_publicacao"].min(), df_negativo["data_publicacao"].max(),

(Timestamp('2022-06-01 13:41:57+0000', tz='UTC'),
 Timestamp('2022-06-10 01:30:38+0000', tz='UTC'))

In [52]:
# separando as colunas pot \t para não dar problemar ao carregar os dados em um novo df

df_positivo.to_csv("raw-data/coleta-positivo-1.csv", sep="\t", index=False)

In [53]:
df_negativo.to_csv("raw-data/coleta-negativo-1.csv", sep="\t", index=False)